# For the Cryo task
- Download a single big DEM that covers all the pairs.
- Need an Excel sheet (CSV) of ALOS-2 data we want to process.
- Start with a small `input.csv` example from Catalina.
- Write a script that will call `alos2_to_rslc.py` on each file, then create a duplicate with the RSLC result.
- Get DEM by running `stage_in.py -b <BOUNDING BOX>` with the coordinates provided by Virginia.
- INSAR runconfg will be `~/alos-to-insar/templates/runconfig_insar_VB_template.yaml`
- Should generate about within 1 to 10 TB orders of magnitude (RSLC, GUNW, RUNW, RIFG, ROFG)

# Length of Time for PCM Jobs
- Cluster start up time: ~8 minutes
- Download time: 1.5 - 2.5 minutes to download each ALOS-2 data file 
- RSLC conversion jobs take 3-5 minutes from my experience (ALOS-1, can't comment extensively on ALOS-2).
- INSAR can vary a decent amount, some jobs will only take ~8 minutes to run, some can take half an hour, it depends on input (this is with GPU).
- 450 acquisitions, 450 pairs
- `2*8 + 450 * (2 + 4/10) + 450 * (15/10)` is `1771 minutes` or about `29.52 hours`. This is assuming we run INSAR only after we finish generating RSLC's.
- RSLC and Downloading Conversion Time Alone is `1088 minutes` or `18.13 hours`.
- INSAR time is `675 minutes` or `11.75 hours`
- So this would take at least around 18 hours, and up to 30 hours.

# Pricing
- At `$3.06/hr` for `p3.2xlarge` instances, the estimated total cost would be `$903.312` for `295.2 hours` of compute time (as the original estimate is reduced by 10) (this included only 2 instances of 8 minute start up time, so there are actually 18 unaccounted for).
- Start up time of 8 minutes means that it costs `$0.408` to start up an instance.
- We get `$910.656` as the total estimated cost after adding in the start up time expenses.

# Login to ASF
This is so that we can download ALOS-1 data if we are missing granules.
Can skip this step if all files are already in S3.

In [8]:
import asf_search as asf
import getpass

session = asf.ASFSession()
username = input('Username:')
password = getpass.getpass('Password:')

try:
    user_pass_session = asf.ASFSession().auth_with_creds(username, password)
except asf.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

Username: max.zhan
Password: ········


Success!


# Raw Input Checker Cell
This cell checks the input files in the `files` list for all unique filename inputs, and whether they are all present in the destination bucket `dest_bucket`.

In [1]:
import os
import csv

import boto3
from urllib.parse import urlparse

dest_bucket = 's3://nisar-st-data-ondemand/ALOS-1-data/20241206/'
alos1_map = {}

files = [
    'alos_20241206.csv',
]
files = [os.path.join('/home/jovyan/alos-to-insar', x) for x in files]

print('Config files:\n', files)

print('Reading into dict...')
for fname in files:
    with open(fname, 'r', encoding='utf-8', newline='') as f:
        for i, row in enumerate(f.readlines()):
            if i == 0:
                continue
            # Add the new granule to the map
            col_where_url_is = 8
            dl_url = row.split(',')[col_where_url_is]
            parsed_url = urlparse(dl_url)
            dl_fname = os.path.basename(parsed_url.path)
            local_fname = os.path.abspath(dl_fname)
            s3_fname = dest_bucket + dl_fname

            granule_name = dl_fname[:dl_fname.rfind('.')]
            alos1_map[granule_name] = {
                'link': dl_url,
                'bucket': s3_fname,
                'basename': dl_fname
            }


# Check S3 for these resources
def list_s3(url: str, prefix: str='') -> list:
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(url)

    ret = []
    for object_summary in my_bucket.objects.filter(Prefix=prefix):
        ret.append(object_summary.key)
    return ret

def s3_url_exists(url: str) -> bool:
    """Returns whether or not the specified path exists on S3."""
    parsed_url = urlparse(url)
    bucket = parsed_url.netloc
    path = parsed_url.path

    if path.startswith('/'):
        path = path[1:]
    if path.endswith('/'):
        path = path[:-1]
    if bucket.endswith('/'):
        bucket = bucket[:-1]

    return path in list_s3(bucket, prefix=path)
    
found = 0
for i, granule in enumerate(alos1_map):
    s3_url = alos1_map[granule]['bucket']
    granule_found = s3_url_exists(s3_url)
    found += granule_found

    # Move it to the bucket if it isn't there
    if not granule_found:
        dl_link = alos1_map[granule]['link']
        dl_dest_dir = '/home/jovyan/download/'
        dl_dest_path = os.path.join(dl_dest_dir, alos1_map[granule]['basename'])
        asf.download_urls(urls=[dl_link], path=dl_dest_dir, session=user_pass_session)
        !set -x; aws s3 cp {dl_dest_path} {s3_url}
        !set -x; rm {dl_dest_path}

        granule_found = s3_url_exists(s3_url)
        found += granule_found
    
dl_coverage = found / len(alos1_map) * 100
print(f'Download coverage: {found} / {len(alos1_map)} ({dl_coverage:.2f}%)')

Config files:
 ['/home/jovyan/alos-to-insar/alos_20241206.csv']
Reading into dict...
Download coverage: 209 / 209 (100.00%)


# RSLC Conversion Cell
This cell checks whether all the granules **identified in the previous cell** are present in the destination bucket. For all **granules which have not been processed**, this cell prints out **the command that would process the remaining granules**.

If no granules are missing, then a completion message will be displayed instead.

In [6]:
import boto3
import math

RSLC_DEST_BUCKET = f'{dest_bucket}RSLC'

def get_bucket_key(url: str) -> bool:
    """Returns the bucket and path of an S3 URL."""
    parsed_url = urlparse(url)
    bucket = parsed_url.netloc
    path = parsed_url.path

    if path.startswith('/'):
        path = path[1:]
    if path.endswith('/'):
        path = path[:-1]
    if bucket.endswith('/'):
        bucket = bucket[:-1]

    return bucket, path


# Adds the RSLC S3 URL to the map
for i, granule in enumerate(alos1_map):
    s3_url = alos1_map[granule]['bucket']
    parsed_url = urlparse(s3_url)
    rslc_fname = os.path.splitext(os.path.basename(parsed_url.path))[0] + '.h5'
    alos1_map[granule]['RSLC'] = f'{RSLC_DEST_BUCKET}/{rslc_fname}'

    
# Summate data volume
s3 = boto3.resource('s3')
data_vol = 0

# Run alos_to_rslc.py
limit = math.inf
count = 0
input_strs = ['']
for i, granule in enumerate(alos1_map):
    if count % limit == 0 and input_strs[-1] != '':
        count = 0
        input_strs.append('')
    s3_url = alos1_map[granule]['bucket']
    rslc_url = alos1_map[granule]['RSLC']
    if s3_url_exists(rslc_url):
        bucket, key = get_bucket_key(rslc_url)
        obj = s3.Object(bucket, key)
        file_size = obj.content_length
        data_vol += file_size
        continue
    if not s3_url_exists(s3_url):
        raise f'Cannot convert granule "{granule}" as the base URL ({s3_url}) does not exist!'
    input_strs[-1] += f'{s3_url} '
    count += 1
if input_strs[-1] == '':
    input_strs = input_strs[:-1]

if len(input_strs) > 0:
    print('Activate Env:\nconda activate ~/otello/.conda')
    cmd = None
    for i, next_str in enumerate(input_strs):
        next_cmd = f'python /home/jovyan/alos-to-insar/alos_to_rslc.py -o {RSLC_DEST_BUCKET} {next_str};'
        if cmd == None:
            cmd = next_cmd
        else:
            cmd += next_cmd + ' '
        print(f'Input {i}:\n\t{next_cmd}')
    print(f'Full Command:\n\t{cmd}')
else:
    print('All done, all ALOS-1 granules have been fully processed.')

data_vol /= 1e9
print(f'{data_vol=} GB')

All done, all ALOS-1 granules have been fully processed.
data_vol=227.608100864 GB


# Save ALOS-2 Map to a JSON to reference later
This cell exists just to let you skip the previous two cells by writing the granules to process to a file.

In [7]:
import json

alos1_map_fname = os.path.join('/home/jovyan/alos-to-insar', 'alos1_map.json')
# Save to json
with open(alos1_map_fname, 'w', encoding='utf-8') as outfile: 
    json.dump(alos1_map, outfile)

# Prepare GCOV Input File
This cell determines the pairs to run from the original input files and sanitizes whether there's a bad pair. It then writes that pair as S3 URLs to a new file which can be passed to `rslc_to_insar.py`.

In [9]:
import boto3
import math

GCOV_DEST_BUCKET = f'{dest_bucket}GCOV-sigma0'


def get_bucket_key(url: str) -> bool:
    """Returns the bucket and path of an S3 URL."""
    parsed_url = urlparse(url)
    bucket = parsed_url.netloc
    path = parsed_url.path

    if path.startswith('/'):
        path = path[1:]
    if path.endswith('/'):
        path = path[:-1]
    if bucket.endswith('/'):
        bucket = bucket[:-1]

    return bucket, path


# Adds the GCOV S3 URL to the map
for i, granule in enumerate(alos1_map):
    s3_url = alos1_map[granule]['RSLC']
    parsed_url = urlparse(s3_url)
    gcov_fname = os.path.splitext(os.path.basename(parsed_url.path))[0] + '_GCOV.h5'
    alos1_map[granule]['GCOV'] = f'{GCOV_DEST_BUCKET}/{gcov_fname}'

    
# Summate data volume
s3 = boto3.resource('s3')
data_vol = 0

# Run alos_to_rslc.py
limit = math.inf
count = 0
input_strs = ['']
for i, granule in enumerate(alos1_map):
    if count % limit == 0 and input_strs[-1] != '':
        count = 0
        input_strs.append('')
    s3_url = alos1_map[granule]['RSLC']
    gcov_url = alos1_map[granule]['GCOV']
    if s3_url_exists(gcov_url):
        bucket, key = get_bucket_key(gcov_url)
        obj = s3.Object(bucket, key)
        file_size = obj.content_length
        data_vol += file_size
        continue
    if not s3_url_exists(s3_url):
        raise f'Cannot convert granule "{granule}" as the base URL ({s3_url}) does not exist!'
    input_strs[-1] += f'{s3_url} '
    count += 1
if input_strs[-1] == '':
    input_strs = input_strs[:-1]

if len(input_strs) > 0:
    print('Activate Env:\nconda activate ~/otello/.conda')
    cmd = None
    for i, next_str in enumerate(input_strs):
        next_cmd = f'python /home/jovyan/alos-to-insar/rslc_to_gcov.py -o {GCOV_DEST_BUCKET} {next_str};'
        if cmd == None:
            cmd = next_cmd
        else:
            cmd += next_cmd + ' '
        print(f'Input {i}:\n\t{next_cmd}')
    print(f'Full Command:\n\t{cmd}')
else:
    print('All done, all ALOS-1 RSLC files have been fully processed.')

data_vol /= 1e9
print(f'{data_vol=} GB')

All done, all ALOS-1 RSLC files have been fully processed.
data_vol=46.64066048 GB


# GCOV Product Completion Checker
Checks how many of the GCOV products were successfully uploaded to the expected S3 URLs and prints out how many pairs are missing.

In [10]:
# Summate data volume
s3 = boto3.resource('s3')
data_vol = 0

found = 0
small_products = 0
mb_threshold = 10
for i, granule in enumerate(alos1_map):
    gcov_url = alos1_map[granule]['GCOV']
    if s3_url_exists(gcov_url):
        bucket, key = get_bucket_key(gcov_url)
        obj = s3.Object(bucket, key)
        file_size = obj.content_length
        if file_size > 1e6 * mb_threshold:
            data_vol += file_size
            found += 1
        else:
            small_products += 1
        continue
    if not s3_url_exists(s3_url):
        raise f'Cannot convert granule "{granule}" as the base URL ({s3_url}) does not exist!'

dl_coverage = found / len(alos1_map) * 100
print(f'GCOV coverage: {found} / {len(alos1_map)} ({dl_coverage:.2f}% ({small_products} products < {mb_threshold} MB))')

data_vol /= 1e9
print(f'{data_vol=} GB')

GCOV coverage: 209 / 209 (100.00% (0 products < 10 MB))
data_vol=46.64066048 GB


# Run NISARQA Code on all the RSLC and GCOV Products
RSLCs take forever to run while GCOV takes like 10 seconds per GCOV, so do all the GCOVs first and wait ~5 hours per ~200 RSLCs...

In [12]:
import os
import papermill as pm

RUN_QA_MAP = {
    'rslc': False,
    'gcov': True,
}

nisarqa_nb = '/home/jovyan/alos-to-insar/run_nisar_qa.ipynb'
output_nb = '/home/jovyan/alos-to-insar/run_nisar_qa-output.ipynb'

qa_out_folder = os.path.join(os.getcwd(), 'qa')
base_drive = '/home/jovyan/test/qa'

for granule_name, url_list in alos1_map.items():
    folder_name = granule_name[:granule_name.rfind('-')]
    dest_folder = os.path.join(base_drive, folder_name)
    rslc_folder = os.path.join(dest_folder, 'RSLC')
    gcov_folder = os.path.join(dest_folder, 'GCOV-sigma0')

    if RUN_QA_MAP['rslc']:
        pm.execute_notebook(
           nisarqa_nb,
           output_nb,
           parameters = dict(product_type='rslc', target_product_s3_url=url_list['RSLC'])
        )
        os.makedirs(rslc_folder, exist_ok=True)
        !mv {qa_out_folder}/* {rslc_folder}

    if RUN_QA_MAP['gcov']:
        pm.execute_notebook(
           nisarqa_nb,
           output_nb,
           parameters = dict(product_type='gcov', target_product_s3_url=url_list['GCOV'])
        )
        os.makedirs(gcov_folder, exist_ok=True)
        !mv {qa_out_folder}/* {gcov_folder}

print('============== <<< SUCCESS >>> ==============')

Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

============== <<< SUCCESS >>> ==============


In [13]:
# Check if all the QA is done

missing = []
for granule_name, url_list in alos1_map.items():
    folder_name = granule_name[:granule_name.rfind('-')]
    dest_folder = os.path.join(base_drive, folder_name)
    rslc_folder = os.path.join(dest_folder, 'RSLC')
    gcov_folder = os.path.join(dest_folder, 'GCOV-sigma0')

    if not (os.path.isdir(rslc_folder) and os.path.isdir(gcov_folder)):
        missing.append(granule_name)

print(f'Missing QA products: {missing}')

Missing QA products: []
